In [6]:
import pandas as pd
import json
import numpy as np

In [7]:
config = json.load(open('../config.json', 'r'))
DATA_DIR = config['DATA_DIR']
TWEET_DIR = config['TWEET_DIR']
NUM_CLUSTERS = config['NUM_CLUSTERS']
events = open(DATA_DIR + 'event_names.txt', 'r').read().splitlines()

In [20]:
#data_filter_method = '_noRT' # make sure to include underscore
data_filter_method = '_clustered'
cluster_method = '_relative'
#cluster_method = ''

In [15]:
time_dict = json.load(open(DATA_DIR + "event_year.json","r"))
shooter_race = json.load(open(DATA_DIR + "shooters_race.json","r"))
polarization_dict = json.load(open(DATA_DIR + "polarization"+data_filter_method+ cluster_method+".json","r"))

In [21]:
between_topic_pol = json.load(open(DATA_DIR + "between_topic_polarization"+cluster_method+".json","r"))

## plotting overall polarization

In [12]:
x = []
y = []
y_random = []
labels=[]
ex = ['fort_lauderdale']  # we exclude Fort Lauderdale because we only have data for the first day after the shooting
for e, t in time_dict.items():
    if e in ex:
        continue
    label = e.split('_')
    new_label = []
    for l in label:
        new_label.append(l[0].upper() + l[1:])
    new_label = ' '.join(new_label)
    
    x_val = float(2000+t)
    y_val = float(polarization_dict[e][0])
    y_random_val = float(float(polarization_dict[e][1]))
    x.append(x_val)
    y.append(y_val)
    y_random.append(y_random_val)
    labels.append(new_label)
    #labels.append(plt.text(x_val, y_val, new_label, fontsize=8))
df = pd.DataFrame.from_dict({'year':np.array(x * 2), 'polarization':np.array(y + y_random), 'label':labels * 2, 'is_actual':['actual value']* len(y) + ['value resulting from random party assignment']* len(y)})
df.to_csv(DATA_DIR + "polarization"+data_filter_method+cluster_method+".csv", index=False)


## plotting within vs between topic polarization

In [22]:
x = []
y_between = []
y_within = []
labels=[]
race = []
ex = ['fort_lauderdale']
for e, t in time_dict.items():
    if e in ex:
        continue
    label = e.split('_')
    new_label = []
    for l in label:
        new_label.append(l[0].upper() + l[1:])
    new_label = ' '.join(new_label)
    
    within_topic_pol = json.load(open(TWEET_DIR +e+"/"+e+"_topic_polarization"+cluster_method+".json","r"))
    
    # get the topic proportions
    topics = np.load(TWEET_DIR + e + '/' + e + '_cluster_labels_' + str(NUM_CLUSTERS) + cluster_method + '.npy')
    print(e, len(topics))
    within_topic_pol_val = (np.array([float(within_topic_pol[str(i)][0]) for i in range(NUM_CLUSTERS)]) * np.bincount(topics)).sum() / len(topics)
    
    x_val = float(2000+t)
    y_between_val = float(between_topic_pol[e][0])
    y_within_val = within_topic_pol_val
    x.append(x_val)
    y_between.append(y_between_val)
    y_within.append(y_within_val)
    labels.append(new_label)
    race.append(shooter_race[e])
    #labels.append(plt.text(x_val, y_val, new_label, fontsize=8))
df = pd.DataFrame.from_dict({'year':np.array(x * 2), 'polarization':np.array(y_between + y_within), 'label':labels * 2, 'kind':['between-topic']* len(y_between) + ['within-topic']* len(y_within), 'race':race * 2})
df.to_csv(DATA_DIR + "topic_polarization"+cluster_method+".csv", index=False)

annapolis 14724
baton_rouge 22806
burlington 4173
chattanooga 16580
colorado_springs 31400
dallas 112359
fresno 4894
kalamazoo 5541
nashville 19999
orlando 648574
parkland 141261
pittsburgh 29072
roseburg 9366
san_bernardino 36324
san_francisco 5369
santa_fe 33721
sutherland_springs 80844
thornton 7411
thousand_oaks 46897
vegas 531495
